In [27]:
import pandas as pd
import json
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, f1_score, precision_score, recall_score
from transformers import pipeline
import torch
import numpy as np
import os 
from datetime import datetime
import json



In [29]:
RANDOM_STATE = 42
path_DATA = "../../../../../data/spanish"

csv_path = f"{path_DATA}/dataset/oficialDatasetEAIM2026.csv"


TESTING = False

if TESTING:
    NROWS = None
else:
    NROWS = None
    


In [30]:
df = pd.read_csv(csv_path, nrows=NROWS)
display(df.head(2))


,_id,artist,genre,lyrics,composer,lyrics_word_count,title_songs_new,spotify_id,popularity,explicit_content,duration_ms,release_date,external_urls.spotify,letras_path,id_yt
0,689a8a83a437eda121bd38ce,Laura Pausini,pop,Ya no responde ni al teléfono\nPende de un hil...,Compuesta por: Federico Cavalli / Angelo Valsi...,445,Se Fue,5oQadhkuEdEhtdVn0QceyZ,71,False,240179,2024-11-15,https://open.spotify.com/track/5oQadhkuEdEhtdV...,Laura Pausini/30278,g-GBiuujmL8
1,689a8a83a437eda121bd389f,Shakira,pop,Loca\n(Loca)\nNo te ponga' bruto\n\nQue te la ...,Compuesta por: Pitbull / El Cata / Shakira / C...,360,Loca (part. El Cata),42k1KeBehAd83lrGt1okiC,76,False,183693,2010-10-19,https://open.spotify.com/track/42k1KeBehAd83lr...,Shakira/1735339,XAhTt60W7qo


In [47]:
# # Split the indexes 
# df = pd.read_csv(csv_path, nrows=NROWS)
# y = df['explicit_content']

# train_idx, test_idx = train_test_split(
#     df.index,
#     test_size=0.2,
#     random_state=42,
#     stratify=y 
# )

# train_indices = df.loc[train_idx, ['_id']].copy()
# train_indices['original_index'] = train_idx

# test_indices = df.loc[test_idx, ['_id']].copy()
# test_indices['original_index'] = test_idx

# train_indices.to_csv(f"{path_DATA}/train_indices_spanish.csv", index=False)
# test_indices.to_csv(f"{path_DATA}/test_indices_spanish.csv", index=False)

# print(f"Train size: {len(train_idx)}")
# print(f"Test size: {len(test_idx)}")

# display(train_indices.head(2))
# display(test_indices.head(2))



In [31]:
path_train_indices = f"{path_DATA}/train_indices_spanish.csv"
path_test_indices = f"{path_DATA}/test_indices_spanish.csv"


df_train_indices = pd.read_csv(path_train_indices, nrows=NROWS)
df_test_indices = pd.read_csv(path_test_indices, nrows=NROWS)

idx_train = df_train_indices['original_index'].values
idx_test = df_test_indices['original_index'].values

display(df_train_indices.head(2))
display(df_test_indices.head(2))

df = pd.read_csv(csv_path, nrows=NROWS)
# display(df)

,_id,original_index
0,689a8a9da437eda121bd54f1,2097
1,689a8ab7a437eda121bd6def,4037


,_id,original_index
0,689a8aafa437eda121bd65ae,2988
1,689a8eaca437eda121bd84fe,6399


# meta-llama/Llama-3.1-8B-Instruct
https://huggingface.co/meta-llama/Llama-3.1-8B-Instruct

In [32]:
import os
import transformers
import torch


model_id = "meta-llama/Meta-Llama-3.1-8B-Instruct"

pipeline = transformers.pipeline(
    "text-generation",
    model=model_id,
    model_kwargs={"torch_dtype": torch.bfloat16},
    device_map="auto",
    pad_token_id=128001
    # pad_token_id` to `eos_token_id`:128001
)



Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

Device set to use cuda:0


In [33]:
X = df['lyrics']
y = df['explicit_content']

X_train = X.loc[idx_train]
X_test = X.loc[idx_test]
y_train = y.loc[idx_train]
y_test = y.loc[idx_test]

# display(X_test.head())


In [34]:
test_scores = []

i_tested = 0
NROWS = 20
answers_debug = []
sons_debug = []
for lyric in X_train:
    messages = [
        {"role": "system", "content": "Eres un experto en detectar si una canción es explícita o no basándote en su letra. Responde ÚNICAMENTE con una palabra: 'Explícita' o 'No-Explícita'."},
        {"role": "user", "content": f"letra de la canción: {lyric}"},
    ]
    output = pipeline(messages, max_new_tokens=64)
    # print()
    # answer = output[0]["generated_text"].strip().split()[-1]
    answer = output[0]["generated_text"][-1]['content']
    if TESTING:
        answers_debug.append(answer)
        sons_debug.append(lyric)
        
    score_explicit = 1 if answer.lower() == "explícita" else 0
    test_scores.append(score_explicit)
    i_tested += 1

    if i_tested == NROWS and TESTING:
        break

if TESTING:
    y_train = y_train[:NROWS]
    idx_train = idx_train[:NROWS]
    


test_scores = np.array(test_scores)


# Métricas
f1_test = f1_score(y_train, test_scores)
accuracy_test = accuracy_score(y_train, test_scores)
precision_test = precision_score(y_train, test_scores)
recall_test = recall_score(y_train, test_scores)


print(f"Accuracy en test: {accuracy_test:.8f}")
print(f"Precision en test: {precision_test:.8f}")
print(f"Recall en test: {recall_test:.8f}")
print(f"F1 en test: {f1_test:.8f}")

resultados = {
    "accuracy": float(f"{accuracy_test:.8f}"),
    "precision": float(f"{precision_test:.8f}"),
    "recall": float(f"{recall_test:.8f}"),
    "f1": float(f"{f1_test:.8f}")
}

if TESTING:
    test_ou = "test"
else:
    test_ou = ""

ruta_resultados = f"resultados_train_llama_8B{test_ou}.json"
with open(ruta_resultados, "w") as f:
    json.dump(resultados, f, indent=4)

print(f"Resultados guardados en {ruta_resultados}")


df_resultados = pd.DataFrame({
    "original_index": idx_train,
    "__id": df.loc[idx_train, "_id"].values if "_id" in df.columns else None,
    "prediccion": test_scores
})

nombre_salida = f"predicciones_train_llama_8B{test_ou}.csv"
df_resultados.to_csv(nombre_salida, index=False)



Accuracy en test: 0.75593510
Precision en test: 0.17735602
Recall en test: 0.61173815
F1 en test: 0.27498732
Resultados guardados en resultados_train_llama_8B.json


In [36]:
test_scores = []

i_tested = 0
NROWS = 20
answers_debug = []
sons_debug = []
for lyric in X_test:
    messages = [
        {"role": "system", "content": "Eres un experto en detectar si una canción es explícita o no basándote en su letra. Responde ÚNICAMENTE con una palabra: 'Explícita' o 'No-Explícita'."},
        {"role": "user", "content": f"letra de la canción: {lyric}"},
    ]
    output = pipeline(messages, max_new_tokens=64)
    # print()
    # answer = output[0]["generated_text"].strip().split()[-1]
    answer = output[0]["generated_text"][-1]['content']
    if TESTING:
        answers_debug.append(answer)
        sons_debug.append(lyric)
        
    score_explicit = 1 if answer.lower() == "explícita" else 0
    test_scores.append(score_explicit)
    i_tested += 1

    if i_tested == NROWS and TESTING:
        break

if TESTING:
    y_train = y_train[:NROWS]
    idx_train = idx_train[:NROWS]
    


test_scores = np.array(test_scores)


# Métricas
f1_test = f1_score(y_test, test_scores)
accuracy_test = accuracy_score(y_test, test_scores)
precision_test = precision_score(y_test, test_scores)
recall_test = recall_score(y_test, test_scores)


print(f"Accuracy en test: {accuracy_test:.8f}")
print(f"Precision en test: {precision_test:.8f}")
print(f"Recall en test: {recall_test:.8f}")
print(f"F1 en test: {f1_test:.8f}")

resultados = {
    "accuracy": float(f"{accuracy_test:.8f}"),
    "precision": float(f"{precision_test:.8f}"),
    "recall": float(f"{recall_test:.8f}"),
    "f1": float(f"{f1_test:.8f}")
}

if TESTING:
    test_ou = "test"
else:
    test_ou = ""

ruta_resultados = f"resultados_test_llama_8B{test_ou}.json"
with open(ruta_resultados, "w") as f:
    json.dump(resultados, f, indent=4)

print(f"Resultados guardados en {ruta_resultados}")


df_resultados = pd.DataFrame({
    "original_index": idx_test,
    "__id": df.loc[idx_test, "_id"].values if "_id" in df.columns else None,
    "prediccion": test_scores
})

nombre_salida = f"predicciones_test_llama_8B{test_ou}.csv"
df_resultados.to_csv(nombre_salida, index=False)



Accuracy en test: 0.76980874
Precision en test: 0.20263158
Recall en test: 0.69369369
F1 en test: 0.31364562
Resultados guardados en resultados_test_llama_8B.json


In [ ]:
if TESTING:
    import textwrap
    print("Answers debug:", answers_debug)
    print("test_scores:", test_scores)
    print("y true     :", y_test.values)

    print("song 3:", sons_debug[2])
    wrapped_song = textwrap.fill(sons_debug[2], width=80)
    print("song 3:")
    print(wrapped_song)